In [1]:
from transformers import pipeline

summarization_model = pipeline("summarization")
sentiment_model = pipeline("sentiment-analysis")
translation_model = pipeline('translation_en_to_fr')
image_classification = pipeline("image-classification")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to t5-base and revision 686f1db (https://huggingface.co/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
/home/quinten/Utilisateurs/slemouellic/.local/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept 

In [4]:
import download_file

def new_predict(sentiment_model,summarization_model, translation_model,image_classification,input_type, data = None,min_length = 0, max_length = 150):
    """
    make and combine the prediction of all the differents models on the differents scoring tables

    params_scoring dictionnary: dictionnary that contain all the scoring tables on which models will...
    ... make predictions. It have to be a dictionnary as the predict function of mlflow only take a single argument.

    Return Pandas dataframe with all the fraud risk score (and few others informations) on the remise batch
    Return interpretation_remises pandas dataframe with the interpretation for the remises model
    Return interpretation_client pandas dataframe with the interpretation for the clients models
    """
    
    if input_type == "sentiment":
        return sentiment_model(data)

    elif input_type == "summarization":

        if data is None:
            data = download_file.download_story()

        dict_result = summarization_model(data, min_length, max_length)[0]
        dict_result["input_text"] = data
        return dict_result

    elif input_type == "translation":
        return translation_model(data)

    elif input_type == "image":
        image = download_file.download_image(data)
        return image_classification(image)

    return "mauvais type selectionné"


In [5]:
import os
import pkgutil
import importlib
import sys
import cloudpickle
import mlflow



def variable_for_value(value):
    for n,v in globals().items():
        if v == value:
            return n
    return None

class custom_ml_flow_model(mlflow.pyfunc.PythonModel):

    def __init__(self):
        return
        
    def load_context(self, context):
        
        import cloudpickle
        import inspect
        
        list_artifacts_name = []
        
        for name, artifact in context.artifacts.items():
            list_artifacts_name.append(name)
            print(name)
            with open(artifact, "rb") as f:
                exec("self.{} =  cloudpickle.load(f)".format(name))
            print("chargé")
                

        signature = inspect.signature(self.__predict__)

        self.arg_default_value = {
                                k: v.default
                                for k, v in signature.parameters.items()
                                if v.default is not inspect.Parameter.empty
                                }

        list_args_predict = [k for k, v in signature.parameters.items()]

        dict_args___predict__ = {}
        
        for arg in list_args_predict:
            if arg in list_artifacts_name:
                dict_args___predict__[arg] = "self.{}".format(arg)
            else :
                dict_args___predict__[arg] = "params_predict[\"{}\"]".format(arg)
                
        self.dict_args___predict__ = dict_args___predict__
        
    def register_all_function_by_value(self):    
        
        sub_folder_names = [(x) for x in os.listdir() if os.path.isdir(x)]
    
        for fold_name, folder in [(None, os.getcwd())] + [(x, os.path.join(os.getcwd(), x)) for x in os.listdir() if os.path.isdir(x)]:
            if ".ipynb_" not in folder:
                for module in [name for _, name, _ in pkgutil.iter_modules([folder]) if name not in sub_folder_names]:
                    print(module)
                    full_module = module if fold_name is None else fold_name + "." + module
                    module_spec = importlib.util.spec_from_file_location(full_module,os.path.join(folder, "{}.py".format(module)))
                    module = importlib.util.module_from_spec(module_spec)
                    sys.modules[module_spec.name] = module
                    module_spec.loader.exec_module(module)
                    cloudpickle.register_pickle_by_value(module)
            
    def pickle_artifacts(self, artifact_path, predict_function, models):
        artifacts = {}
        
        name_pred = "__predict__"
        path_pred = os.path.join(artifact_path, '{}.pkl'.format(name_pred))
        with open(path_pred, "wb") as f:
            cloudpickle.dump(predict_function, f)
            
            artifacts[name_pred] = path_pred

        for model in models:
            name = variable_for_value(model)
            path = os.path.join(artifact_path, "{}.pkl".format(name))
            with open(path, "wb") as f:
                cloudpickle.dump(model, f)
                
            artifacts[name] = path
            
        return artifacts
        
                
    def save(self, artifact_path, predict_function, models):
        
        self.register_all_function_by_value()
        
        artifacts = self.pickle_artifacts(artifact_path, predict_function, models)
                
        mlflow.pyfunc.log_model(
            artifact_path = "Model",
            python_model   =  self,
            artifacts      =  artifacts)
        
        print(artifacts)
        
        for name, inter_model in artifacts.items():
            os.remove(artifacts[name])
                            
    def predict(self, context, params_predict): 
        """
        make and combine the prediction of all the differents models on the differents scoring tables
    
        params_scoring dictionnary: dictionnary that contain all the scoring tables on which models will...
        ... make predictions. It have to be a dictionnary as the predict function of mlflow only take a single argument.
    
        Return Pandas dataframe with all the fraud risk score (and few others informations) on the remise batch
        Return interpretation_remises pandas dataframe with the interpretation for the remises model
        Return interpretation_client pandas dataframe with the interpretation for the clients models
        """
        
        for arg in self.arg_default_value:
            if arg not in params_predict.keys():
                params_predict[arg] = self.arg_default_value[arg]
        
        new_dict = {}
        
        for x, y in self.dict_args___predict__.items():
            new_dict[x] = eval(y)  
            
        return self.__predict__(**new_dict)


In [6]:
tracking_uri ="/home/quinten/Utilisateurs/slemouellic/New_test_ml_flow/test_artifacts/mlruns/"
experiment_id = "quinten" 

mlflow.set_tracking_uri(tracking_uri)        
mlflow_experiment_path = os.path.join(tracking_uri,experiment_id)

with mlflow.start_run(experiment_id = experiment_id) as run:
        mlruns_artifcats_path = os.path.join(mlflow_experiment_path, run.info.run_id)
        print(mlruns_artifcats_path)
        
        model = custom_ml_flow_model()
        
        model.save(artifact_path = mlruns_artifcats_path,
                    predict_function = new_predict, 
                   models = [summarization_model, 
                             image_classification,
                            translation_model,
                            sentiment_model])

/home/quinten/Utilisateurs/slemouellic/New_test_ml_flow/test_artifacts/mlruns/quinten/84ea04c00aee411bb3905607782786ad
download_file
no_import
to_import
utils
test_import
{'__predict__': '/home/quinten/Utilisateurs/slemouellic/New_test_ml_flow/test_artifacts/mlruns/quinten/84ea04c00aee411bb3905607782786ad/__predict__.pkl', 'summarization_model': '/home/quinten/Utilisateurs/slemouellic/New_test_ml_flow/test_artifacts/mlruns/quinten/84ea04c00aee411bb3905607782786ad/summarization_model.pkl', 'image_classification': '/home/quinten/Utilisateurs/slemouellic/New_test_ml_flow/test_artifacts/mlruns/quinten/84ea04c00aee411bb3905607782786ad/image_classification.pkl', 'translation_model': '/home/quinten/Utilisateurs/slemouellic/New_test_ml_flow/test_artifacts/mlruns/quinten/84ea04c00aee411bb3905607782786ad/translation_model.pkl', 'sentiment_model': '/home/quinten/Utilisateurs/slemouellic/New_test_ml_flow/test_artifacts/mlruns/quinten/84ea04c00aee411bb3905607782786ad/sentiment_model.pkl'}
